In [41]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator

In [52]:
# run params
section = 'vae'
run_id = '0012'
data_name = 'Greens'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = 'data\Green_MIP'

In [53]:
#print(os.path.join(DATA_FOLDER, '*\*.tif'))
glob(os.path.join(os.getcwd(), 'data\Green_MIP\*.tif')) 
#print(os.path.join(os.getcwd(), 'data\Green_MIPs'))
#data_gen.flow_from_directory('data\Green_MIPs')
#if mode == 'build':
 #   vae.save('0010_Green')

[]

In [54]:
INPUT_DIM = (40,40,1)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(os.getcwd(),'data\Green_MIPs\*.tif')))
filenames2 = np.array(glob(os.path.join(os.getcwd(),'data\Green_MIP\Red_MIP\*.tif')))

NUM_IMAGES = len(filenames)+len(filenames2)
print(filenames)

['C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_10.tif'
 'C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_100.tif'
 'C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_1000.tif'
 ...
 'C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_9997.tif'
 'C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_9998.tif'
 'C:\\Users\\zach101\\Documents\\GDL Book\\GDL_code-master\\GDL_code-master\\data\\Green_MIPs\\mip_9999.tif']


In [55]:
data_gen = ImageDataGenerator(rescale=1./65535)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , color_mode = 'grayscale'
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 19998 images belonging to 2 classes.


In [56]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[1,2,2,1]
                , decoder_conv_t_filters=[64,64,32,1]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[1,2,2,1]
                , z_dim=50
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [57]:
vae.encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 40, 40, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 40, 40, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 40, 40, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_29 (LeakyReLU)      (None, 40, 40, 32)   0           batch_normalization_29[0][0]     
__________________________________________________________________________________________________
dropout_29

In [58]:
vae.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6400)              326400    
_________________________________________________________________
reshape_5 (Reshape)          (None, 10, 10, 64)        0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 10, 10, 64)        36928     
_________________________________________________________________
batch_normalization_33 (Batc (None, 10, 10, 64)        256       
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 10, 10, 64)        0         
__________

In [59]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 40
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [60]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [61]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/40
625/624 [==============================] - 147s 235ms/step - loss: 214.5113 - vae_r_loss: 184.2163 - vae_kl_loss: 30.2950

Epoch 00001: saving model to run/vae/0012_Greens\weights/weights-001-214.53.h5

Epoch 00001: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 2/40
625/624 [==============================] - 144s 231ms/step - loss: 9.0207 - vae_r_loss: 1.7642 - vae_kl_loss: 7.2565

Epoch 00002: saving model to run/vae/0012_Greens\weights/weights-002-9.02.h5

Epoch 00002: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 3/40
625/624 [==============================] - 146s 234ms/step - loss: 4.3206 - vae_r_loss: 0.5941 - vae_kl_loss: 3.7265

Epoch 00003: saving model to run/vae/0012_Greens\weights/weights-003-4.32.h5

Epoch 00003: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 4/40
625/624 [==============================] - 144s 231ms/step - loss: 2.4019 - vae_r_loss: 0.2924 - vae_kl_loss: 2.1095

Epoch 00004: saving model to run/vae/00

625/624 [==============================] - 143s 229ms/step - loss: 0.0521 - vae_r_loss: 0.0164 - vae_kl_loss: 0.0356

Epoch 00031: saving model to run/vae/0012_Greens\weights/weights-031-0.05.h5

Epoch 00031: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 32/40
625/624 [==============================] - 143s 229ms/step - loss: 0.0202 - vae_r_loss: 0.0159 - vae_kl_loss: 0.0042

Epoch 00032: saving model to run/vae/0012_Greens\weights/weights-032-0.02.h5

Epoch 00032: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 33/40
625/624 [==============================] - 143s 229ms/step - loss: 0.0167 - vae_r_loss: 0.0156 - vae_kl_loss: 0.0011

Epoch 00033: saving model to run/vae/0012_Greens\weights/weights-033-0.02.h5

Epoch 00033: saving model to run/vae/0012_Greens\weights/weights.h5
Epoch 34/40
625/624 [==============================] - 143s 229ms/step - loss: 0.0162 - vae_r_loss: 0.0154 - vae_kl_loss: 8.1684e-04

Epoch 00034: saving model to run/vae/0012_Greens\w